In [1]:
!pip -q install git+https://github.com/huggingface/transformers # often need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install langchain
!pip install -q -U git+https://github.com/huggingface/peft.git
# pip install -q -U git+https://github.com/huggingface/accelerate.git


In [3]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [4]:
torch.set_default_device('cuda')

In [5]:
model_name = 'teknium/OpenHermes-2.5-Mistral-7B'

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             torch_dtype="auto").eval()

tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          use_fast=False,
                                          torch_dtype="auto")



config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/101 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# Prompt content: "hi"
messages = [
    {"role": "system", "content": "you are a helpful assistant who is a research master"},
    {"role": "user", "content": "hi"}
]

In [30]:
tokenizer.pad_token = tokenizer.eos_token



In [64]:
model_name = 'Intel/neural-chat-7b-v3-1'

model_intel = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             torch_dtype="auto").eval()

tokenizer_intel = AutoTokenizer.from_pretrained(model_name,
                                          use_fast=False,
                                          torch_dtype="auto")



config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/145 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [67]:
tokenizer_intel.pad_token = tokenizer_intel.eos_token


In [65]:
def generate_intel(input_text, system_prompt="",max_length=512):
    messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": input_text }
    ]
    input_ids = tokenizer_intel.apply_chat_template(conversation=messages,
                                          tokenize=True,
                                          add_generation_prompt=True,
                                          return_tensors='pt')

    output_ids = model_intel.generate(input_ids.to('cuda'), max_length=max_length)

    response = tokenizer_intel.decode(output_ids[0][input_ids.shape[1]:],
                                skip_special_tokens=True)
    wrapped_text = wrap_text(response)
    print(wrapped_text)



### ML Chat Prompt Format
```
messages = [
    {"role": "system", "content": "you are a helpful assistant who is a research master"},
    {"role": "user", "content": "hi"},
    {"role": "assistant", "content": "Hello! How can I assist you today? If you have any questions or need information on a specific topic, feel free to ask."}
]
```

In [35]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [50]:
def generate(input_text, system_prompt="",max_length=512):
    messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": input_text }
    ]
    input_ids = tokenizer.apply_chat_template(conversation=messages,
                                          tokenize=True,
                                          add_generation_prompt=True,
                                          return_tensors='pt')

    output_ids = model.generate(input_ids.to('cuda'), max_length=max_length)

    response = tokenizer.decode(output_ids[0][input_ids.shape[1]:],
                                skip_special_tokens=True)
    wrapped_text = wrap_text(response)
    print(wrapped_text)



## Instruction Answering

In [70]:
%%time
generate('Write a detailed analogy between mathematics and a music.',
         max_length=256)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


Mathematics and music are two seemingly distinct disciplines, yet they share a profound
connection that can be likened to a beautiful symphony. Just as a composer weaves together
various notes and melodies to create a harmonious piece, mathematicians use a set of rules
and principles to construct intricate patterns and structures.

Consider the staff of a musical score as the foundation of mathematics, upon which all
other elements are built. Each line and space on the staff represents a specific note,
much like how the basic principles of arithmetic form the basis for more complex
mathematical concepts. Just as a musician must understand the relationships between
different notes to create a cohesive melody, a mathematician must grasp the connections
between various mathematical concepts to solve problems and prove theorems.

The notes within a musical composition can be thought of as individual mathematical
concepts. When combined in specific ways, these notes create chords, which are

In [68]:
%%time
generate_intel('Write a detailed analogy between mathematics and a music.',
         max_length=256)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




Mathematics and music are two distinct yet interconnected fields that share several
similarities in their structure, principles, and the way they are perceived by the human
mind. Both disciplines involve the creation, organization, and appreciation of complex
patterns, which can be analyzed and understood through various methods.

1. Structure: Both mathematics and music are built upon a foundation of rules and
principles that govern their structure. In mathematics, these rules are often referred to
as axioms, while in music, they are known as the rules of harmony, melody, and rhythm.
These rules provide the framework for the creation of new ideas and patterns, allowing for
the development of intricate and sophisticated structures.

2. Patterns and Symmetry: Mathematics and music both rely on the creation and appreciation
of patterns and symmetries. In mathematics, these patterns can be seen in the form of
geometric shapes, number sequences, and algebraic equations. In music, pattern

In [53]:
%%time
generate('What is the difference between a Llama, Vicuna and an Alpaca?',
         max_length=512)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


Llamas, Vicunas, and Alpacas are all members of the camelid family, native to South
America. They have distinct characteristics that set them apart from one another.

1. Llamas:
- Llamas are the largest of the three, standing up to 4 feet tall at the shoulder and
weighing up to 400 pounds.
- They have long ears, a shaggy coat, and are often used as pack animals by humans.
- Llamas are known for their intelligence and are often used as guard animals to protect
other livestock.

2. Vicunas:
- Vicunas are the smallest of the three, standing up to 3.5 feet tall at the shoulder and
weighing up to 200 pounds.
- They have long, thin necks, and their coat is much finer and softer than that of llamas
or alpacas.
- Vicunas are wild animals and are known for their speed and agility. Their wool is highly
valued and is used to make high-quality textiles.

3. Alpacas:
- Alpacas are similar in size to llamas, standing up to 3.5 feet tall at the shoulder and
weighing up to 200 pounds.
- They have shor

In [69]:
%%time
generate_intel('What is the difference between a Llama, Vicuna and an Alpaca?',
         max_length=512)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




Llamas, Vicunas, and Alpacas are all members of the Camelidae family, which also includes
camels. They are all native to South America. The main differences between them are their
size, fur, and use.

1. Size: Llamas are the largest of the three, with an average height of around 6 feet and
a weight of around 300 pounds. Vicunas are smaller, with an average height of around 4
feet and a weight of around 100 pounds. Alpacas are the smallest, with an average height
of around 3 feet and a weight of around 150 pounds.

2. Fur: Llamas have a woolly coat, which is used for making ropes and other items. Vicunas
have a soft, fine, and valuable fur that is used for making high-quality garments. Alpacas
have a soft, luxurious fleece that is also used for making clothing and other items.

3. Use: Llamas are primarily used as pack animals, carrying loads on their backs. They are
also known for their gentle nature and are often kept as pets. Vicunas are primarily used
for their valuable fur, which

In [54]:
%%time
generate('Write a short email to Sam Altman giving reasons to open source GPT-4',
         max_length=512)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


Subject: The Benefits of Open Sourcing GPT-4

Dear Sam Altman,

I hope this email finds you well. I am writing to express my strong belief that open
sourcing GPT-4 would be a significant step forward for the advancement of artificial
intelligence and its applications. I would like to share a few reasons why I believe this
is the case.

1. Collaboration: Open sourcing GPT-4 would allow the global AI community to collaborate
and build upon the technology. This would lead to faster innovation, as researchers and
developers from various backgrounds and organizations could work together to improve the
model, identify new use cases, and address potential challenges.

2. Transparency: By making GPT-4 open source, we would be promoting transparency in AI
development. This would help to alleviate concerns about the "black box" nature of AI
models and allow for a better understanding of how the technology works. This, in turn,
would lead to increased trust in AI applications and their potential 

In [71]:
%%time
generate_intel('Write a short email to Sam Altman giving reasons to open source GPT-4',
         max_length=512)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




Subject: Urgent Request to Open Source GPT-4 for the Benefit of All

Dear Sam Altman,

I hope this email finds you well. As a strong advocate for open-source technology, I
believe that the release of GPT-4, the latest and most advanced version of OpenAI's
language model, should be made available to the public. Here are a few reasons why I
believe this would be beneficial for everyone:

1. Collaboration and Innovation: By making GPT-4 open source, developers and researchers
worldwide can collaborate on its improvement, leading to faster advancements in AI
technology. This will ultimately benefit society as a whole, as we can collectively work
towards addressing challenges and creating new applications.

2. Transparency and Accountability: Open-sourcing GPT-4 will allow for increased
transparency in the development process. This will help to address concerns about bias and
ethical issues that may arise from the use of AI. By making the code publicly available,
researchers and developer

In [55]:
%%time
generate('What is the capital of England?',
         max_length=256)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


The capital of England is London.
CPU times: user 798 ms, sys: 0 ns, total: 798 ms
Wall time: 793 ms


In [72]:
%%time
generate_intel('What is the capital of England?',
         max_length=256)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


London
CPU times: user 202 ms, sys: 1.52 ms, total: 203 ms
Wall time: 200 ms


## GSM8K

In [57]:
generate('Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?',
         max_length=256)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


Step 1: The cafeteria started with 23 apples.
Step 2: They used 20 apples for lunch, so they have 23 - 20 = 3 apples left.
Step 3: They bought 6 more apples, so they now have 3 + 6 = 9 apples in total.


In [73]:
generate_intel('Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?',
         max_length=256)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




Step 1: Find out how many apples were left after lunch.
23 apples (initial) - 20 apples (used for lunch) = 3 apples left

Step 2: Add the 6 apples that were bought.
3 apples (left) + 6 apples (bought) = 9 apples

So, they have 9 apples now.


In [61]:
generate("Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?",
         max_length=512)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


Weng earns $12 per hour and did 50 minutes of babysitting. To find out how much she
earned, we first need to convert the 50 minutes to hours. There are 60 minutes in an hour,
so 50 minutes is equal to 50/60 = 0.8333 hours. Now, we can calculate her earnings by
multiplying her hourly rate by the number of hours she worked: $12 * 0.8333 = $10.

Weng earned $10 for 50 minutes of babysitting.


In [74]:
generate_intel("Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?",
         max_length=512)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




To find out how much Weng earned, we need to calculate the total time she spent
babysitting and then multiply that by her hourly rate.

Step 1: Convert 50 minutes to hours
50 minutes = 50 / 60 = 0.8333333333333333 hours

Step 2: Calculate the total time spent babysitting
Total time = 0.8333333333333333 hours (50 minutes)

Step 3: Multiply the total time by her hourly rate
Earnings = Total time * Hourly rate
Earnings = 0.8333333333333333 * $12
Earnings = $9.999999999999996

Weng earned $9.999999999999996 for babysitting yesterday.


## CodeGen

In [59]:
generate('''```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """''', system_prompt="You are a genius python coder")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


Here's a Python function that prints all prime numbers between 1 and n:

```python
def print_prime(n):
    for num in range(2, n+1):
        if all(num % i != 0 for i in range(2, num)):
            print(num)

# Example usage:
print_prime(50)
```

This function uses a list comprehension and the `all()` function to check if a number is
divisible by any number between 2 and itself. If it's not divisible by any of these
numbers, it's a prime number and gets printed.


In [76]:
generate_intel('''```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """''')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



   for i in range(2, n):
       if is_prime(i):
           print(i)

def is_prime(n):
   """
   Check if a number is prime
   """
   if n <= 1:
       return False
   for i in range(2, n):
       if n % i == 0:
           return False
   return True

print_prime(10)
```

Output:
2 3 5 7


In [60]:
generate('''```python
def detect_prime(n):
   """
   detect if a number is a prime number or not. return True or False
   """''')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


def detect_prime(n):
    if n <= 1:
        return False
    for i in range(2, n):
        if n % i == 0:
            return False
    return True
```


In [77]:
generate_intel('''```python
def detect_prime(n):
   """
   detect if a number is a prime number or not. return True or False
   """''')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



   if n <= 1:
       return False
   for i in range(2, n):
       if n % i == 0:
           return False
   return True

print(detect_prime(11))
print(detect_prime(13))
print(detect_prime(17))
print(detect_prime(19))
print(detect_prime(23))
print(detect_prime(29))
print(detect_prime(31))
print(detect_prime(37))
print(detect_prime(41))
print(detect_prime(43))
print(detect_prime(47))
print(detect_prime(53))
print(detect_prime(59))
print(detect_prime(61))
print(detect_prime(67))
print(detect_prime(71))
print(detect_prime(73))
print(detect_prime(79))
print(detect_prime(83))
print(detect_prime(89))
print(detect_prime(97))
print(detect_prime(101))
print(detect_prime(103))
print(detect_prime(107))
print(detect_prime(109))
print(detect_prime(113))
print(detect_prime(127))
print(detect_prime(131))
print(detect_prime(137))
print(detect_prime(139))
print(detect_prime(149))
print(detect_prime(151))
print(detect_prime(157))
print(detect_prime(163))
print(detect_prime(167))
print(detect_prime(173))

In [62]:
def detect_prime(n):
    if n <= 1:
        return False
    for i in range(2, n):
        if n % i == 0:
            return False
    return True

detect_prime(37)

True